# ChIP-Seq analysis notebook

This interactive computational notebook is a template for analyzing ChIP-seq data preprocessed by BWA, MACS2, MEME, and MAST, performed by a companion upstream pipeline. 

To get started, put your processed pipeline outputs in the `data` folder.

## Setup the environment
Set the name of the experiment below. "E-1" is the name of the test experiment, using sample data from PPARGC1A ChIP-Seq in HepG2 generated by [Charos et al.](https://www.ncbi.nlm.nih.gov/pubmed/22955979)

In [1]:
expt_id = 'E-1'

Let's next load all the necessary libraries, and check that the data can be loaded.

In [2]:
# Load packages and set up notebook environment
options(warn = -1)
options(jupyter.plot_mimetypes = 'image/png')
suppressPackageStartupMessages(require(tidyverse))

Conflicts with tidy packages ---------------------------------------------------


In [ ]:
# Unzip the data
unzip(zipfile, overwrite=TRUE, exdir='data')

In [ ]:
# Load MACS2 peak data
fpath = paste0('data/macs2_output/', expt_id, '_peaks.sorted.tsv')
peaks = read.csv(fpath, sep='\t', header=F, 
                 col.names=c('chrom', 'start', 'end', 'length', 'abs_summit', 'pileup', 
                             'log10p', 'fold_enrichment', 'log10q', 'name'))
head(peaks)

## Histogram of peak scores
Stuart recommended two histogram plots as quality controls. The first displays the number of peaks by the peak score. Most peaks should have low scores, with a "long tail" of high-scoring peaks that represent the biologically relevant binding events.

This overall peak score histogram is plotted below.

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)
ggplot(peaks, aes(x=log10q)) + geom_histogram(binwidth=1) + 
    ggtitle('Figure 1: Histogram of peak scores') +
    xlab('-log10(q-value)') + ylab('number of peaks')

## TSS coverage profiles
The ChIPseeker package (vignette [here](http://bioconductor.org/packages/release/bioc/vignettes/ChIPseeker/inst/doc/ChIPseeker.html)
) was used to produce the ChIP-seq-specific plots below, such as a coverage heatmap relative to transcription start sites (TSS) across the genome, and to perform GO enrichment of nearby genes.

Load the packages and read the processed peaks. Peak processing was performed in the pipeline stage, following the steps recommended by Stuart:
- Filter blacklisted regions
- Filter non-canonical contigs (i.e., not chromosomes 1-22/X/Y/Mt) 
- Sort by coverage
- Extend by 100bp on each side
- Take top 300 peaks (by coverage)

In [ ]:
suppressPackageStartupMessages(library('ChIPseeker'))
suppressPackageStartupMessages(library('TxDb.Hsapiens.UCSC.hg38.knownGene'))
suppressPackageStartupMessages(library('org.Hs.eg.db'))

In [ ]:
peak <- readPeakFile('data/macs2_output/E-1_peaks.sorted.filtered.extended.bed')
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

First, plot the peak coverage as a heatmap centered on promoter locations. This plot provides a granular view of all peak tags within 3kb of a promoter.

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
promoter <- getPromoters(TxDb=txdb, upstream=3000, downstream=3000)
tagMatrix <- getTagMatrix(peak, windows=promoter)
tagHeatmap(tagMatrix, xlim=c(-3000, 3000), color="red")

The tag matrix (tag x position relative to promoter) can also be summarized by total coverage at each position.

In [ ]:
options(repr.plot.height=4, repr.plot.width=6)
plotAvgProf(tagMatrix, xlim=c(-3000, 3000),
            xlab="Genomic Region (5'->3')", ylab = "Read Count Frequency")

## Annotate genes and gene regions
Now that we know that binding sites are centered on promoters, as expected, we can search for patterns of enrichment in the gene annotations for these bound genes. ChIPseeker provides an annotation function using the EntrezGene (i.e., RefSeq) gene model to assign the nearest gene features to each peak.

In [ ]:
peak_annots <- annotatePeak('data/macs2_output/E-1_peaks.sorted.filtered.extended.bed', tssRegion=c(-3000, 3000),
                         TxDb=txdb, annoDb="org.Hs.eg.db")

peak_annots_df <- as.data.frame(peak_annots)
head(peak_annots_df)

We can write this out to comma-separated format for opening with Excel:

In [ ]:
write.csv(peak_annots_df, file='peak_annots.csv')

The raw gene feature annotations can be viewed as a pie chart, showing the percentage of peaks landing on promoters, versus other gene features. As expected, promoters and intergenic regions are the dominant feature since this experiment uses transcription factors. This plot may be useful for characterizing the differences in binding patterns between TF and histones.

In [ ]:
options(repr.plot.height=5, repr.plot.width=7)
plotAnnoPie(peak_annots)

The Gene Ontology annotations can be assigned and searched for enrichment in the group of bound genes, compared to their frequency in the genome. The [clusterProfiler](http://bioconductor.org/packages/release/bioc/vignettes/clusterProfiler/inst/doc/clusterProfiler.html)
package can generate enrichment scores from a list of Entrez Gene IDs derived from the peak annotation step.

In [ ]:
# GO analysis using clusterProfiler
suppressPackageStartupMessages(library(clusterProfiler))
gene_ids <- as.data.frame(peak_annots)$geneId
gene <- bitr(gene_ids, fromType='ENTREZID', toType='SYMBOL', OrgDb='org.Hs.eg.db')


First, we extract Biological Process (BP) annotations, which are generally the most informative. We can manually inspect the most frequently occurring annotations, but this will just give us the most general (therefore the most common) annotations, not the ones that are more common than expected.

In [ ]:
ggo <- groupGO(gene     = gene_ids,
               OrgDb    = org.Hs.eg.db,
               ont      = "BP",
               level    = 3,
               readable = TRUE)
cat('Number of annotations')
nrow(ggo)
cat('Top annotation counts')
ggo %>% as.data.frame %>% arrange(desc(Count)) %>% head

More informative are enriched annotations. We can use enrichGO to find which of these annotations are statistically significant by the hypergeometric test. Heat shock protein annotations bubble to the top, with highly significant q-values (i.e., Benjamini-Hochberg FDR adjusted p-values).

In [ ]:
ego <- enrichGO(gene          = gene_ids,
                OrgDb         = org.Hs.eg.db,
                ont           = "BP",
                pAdjustMethod = "BH",
                readable      = TRUE)
head(ego)

Let's write these to a `csv` file as well. Check in the `notebooks` directory for this Excel-readable file.

In [ ]:
write.csv(ego, file='ego.csv')

## MAST motif quantification
As part of the results zipfile created by the pipeline, there is an html file of the MAST algorithm output. This algorithm uses the top motifs returned by MEME, and quantifies their occurrences (hits) in the raw alignments. 

Stuart recommended plotting the distribution of hit counts versus p-value of significance of the hit, for each motif. Here we parse the XML-formatted file provided by MAST, and extract the hit counts for each motif, numbered 0 to 2 (in the order displayed in the MEME report).

In [ ]:
library(XML)
mast_xml <- xmlParse('data/mast_output/mast.xml')

mast_xml_to_hits <- function(mast_xml) {
  data <- xmlToList(mast_xml)
  
  # Dig into the tree with rbind and lapply to extract a data.frame
  df <- 
    do.call('rbind', lapply(data$sequences, function(seq) {
      do.call('rbind', lapply(seq[names(seq) == 'seg'], function(seg) {
        do.call('rbind', lapply(seg[names(seg) == 'hit'], function(hit) {
          return(data.frame(
            motif=as.integer(hit['idx']), 
            pvalue=as.numeric(hit['pvalue'])
          ))
        }))
      }))
    }))
  
  rownames(df) <- NULL
  return(df)
}

In [ ]:
options(repr.plot.height=4, repr.plot.width=6)
df <- mast_xml_to_hits('data/mast_output/mast.xml')
ggplot(df, aes(x=-log10(pvalue))) + geom_histogram(binwidth=0.25) + facet_wrap(~motif)